In [8]:
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [9]:
churn_model=load_model('churn_model.h5')

# loading the encoders and scalers
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('churn_scaler.pkl','rb') as file:
    scaler=pickle.load(file)

with open('onehotencoder_geography.pkl','rb') as file:
    onehotencoder_geography=pickle.load(file)

In [10]:
# Example input data
example_churn_input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [11]:
print(type(onehotencoder_geography))


<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [13]:
# One-hot encode 'Geography'
geo_encoded = onehotencoder_geography.transform([[example_churn_input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehotencoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df.head()

/Users/dhruvyellanki/Documents/Projects/churn_modelling__ANN_project/churn_venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_df = pd.DataFrame([example_churn_input_data])
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [19]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
# Scaling input
input_scaled=scaler.transform(input_df)


In [23]:
#prediction
prediction=churn_model.predict(input_scaled)

1/1 [==============================] - 0s 11ms/step


In [25]:
prediction_probability=prediction[0][0]
if prediction>0.5:
    print("customer is likely to churn")
else:
    print("customer is not likely to churn")

customer is not likely to churn
